In [15]:
# import nemo.collections.asr as nemo_asr
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import numpy as np

In [16]:
import pandas as pd
# from moviepy.editor import VideoFileClip, clips_array
from pathlib import Path

df = pd.read_csv("../data/piracy_val.csv")
df

,Unnamed: 0,ID_piracy,segment,ID_license,segment.1
0,0,ydcrodwtz3mstjq1vhbdflx6kyhj3y0p.mp4,1539-1685,ded3d179001b3f679a0101be95405d2c.mp4,546-692
1,1,t9j5gg42w6s7f62uybhtc51aj5nq9vxd.mp4,158-222,f4b1fd188fe77f9f56de07e867128b13.mp4,392-456
2,2,zui6ud3solfndyetlralstpcjmv3br1f.mp4,139-263,aa16953bbb8fdb2dfc5fc368f4abe89a.mp4,656-780
3,3,aozskym3m7s9ibw5xpgca7m5jqe240vs.mp4,306-449,3219f476e4bc29f6f420e4e6b05ae8bb.mp4,399-542
4,4,3y78kqwdr5lbtsy9tb76n5ylbcrainsx.mp4,517-590,cb718f23524eb08af2f8036eedc6b50c.mp4,119-192
...,...,...,...,...,...
133,133,4ojku252jnbtv04wtnvm73x08xyftpgy.mp4,466-632,fd316c0e150c30ba2156f82e65bfea62.mp4,322-488
134,134,iflvtrjcx50j1kgbb3zgbvpql6zm2gry.mp4,196-471,a808f0de343faf6320bcc94ea57c5381.mp4,462-737
135,135,ztbk9ua8hx1u6l3ol3q5ehh8ugxqlrpd.mp4,165-213,ce9697edebdbec509944e8a010d91d0e.mp4,59-107
136,136,dtpq232karpm09ldxucjqip32hfblrkz.mp4,98-231,8cc8785efca469cdb6fb6efdcd3421d2.mp4,227-360


In [17]:
vidoses = df["ID_piracy"].values[:5].tolist() + df["ID_license"].values[:5].tolist()

In [18]:
df[df["ID_piracy"] == "ydcrodwtz3mstjq1vhbdflx6kyhj3y0p.mp4"]

,Unnamed: 0,ID_piracy,segment,ID_license,segment.1
0,0,ydcrodwtz3mstjq1vhbdflx6kyhj3y0p.mp4,1539-1685,ded3d179001b3f679a0101be95405d2c.mp4,546-692


In [49]:
response = requests.post(
    'http://localhost:8123/upload',
    json={
        "filename": "ydcrodwtz3mstjq1vhbdflx6kyhj3y0p.mp4"
    }
)

print(response.content)

KeyboardInterrupt: 

In [28]:
import requests

for video_id in vidoses:
    response = requests.post(
        'http://localhost:8123/upload',
        json={
            "filename": f"{video_id}"
        }
    )

    print(response.content)

b'Internal Server Error'
b'Internal Server Error'
b'Internal Server Error'
b'Internal Server Error'
b'Internal Server Error'
b'Internal Server Error'
b'Internal Server Error'
b'Internal Server Error'
b'Internal Server Error'
b'Internal Server Error'


In [11]:
from pymilvus import MilvusClient
client = MilvusClient( 
    "http://localhost:19530"
)

In [115]:
paths = {}

for i in range(0, 1700, 2):
    embedding = client.query(
        collection_name="audio_segments_ausil",
        filter = f"video_id == 'ydcrodwtz3mstjq1vhbdflx6kyhj3y0p.mp4' and segment_start == {i}"
    )
    embedding = embedding[0]["embedding"]
    data = client.search(
        collection_name="audio_segments_ausil",
        data=[embedding],
        output_fields=["id", "video_id", "segment_start"],
        limit=30
    )[0]


    search_result = {}
    for item in data:
        video_id = item['entity']['video_id']
        segment_start = item['entity']['segment_start']
        
        if video_id not in search_result:
            search_result[video_id] = []

        search_result[video_id].append(segment_start)

    search_result = {k: sorted(v) for k, v in search_result.items()}
    
    for video_id, candidates in search_result.items():
        if video_id not in paths:
            paths[video_id]  =  []
        for candidate in candidates:
            used = False
            for idx, path in enumerate(paths[video_id]):
                if candidate - path[-1] < 10 and candidate - path[-1] > 0:
                    paths[video_id][idx].append(candidate)
                    used = True
                    break
            if not used:
                paths[video_id].append([candidate])

In [117]:
def merge_intervals(intervals):
    if not intervals:
        return []

    # Сортируем интервалы по первому элементу каждого интервала
    intervals.sort(key=lambda x: x[0])

    merged = [intervals[0]]
    for current in intervals[1:]:
        last = merged[-1]
        if current[0] <= last[-1]:  # Проверяем пересечение
            # Объединяем интервалы
            merged[-1] = sorted(list(set(last + current)))
        else:
            merged.append(current)

    return merged

In [118]:
paths = {
    k: [interval for interval in v if len(interval)>1]
    for k, v in paths.items()
}

paths = {
    k: sorted(merge_intervals(v), key=len, reverse=True)[:3] for k, v in paths.items()
}

In [119]:
paths

{'ydcrodwtz3mstjq1vhbdflx6kyhj3y0p.mp4': [[0,
   2,
   4,
   6,
   8,
   10,
   12,
   14,
   16,
   18,
   20,
   22,
   24,
   26,
   28,
   30,
   32,
   34,
   36,
   38,
   40,
   42,
   44,
   46,
   48,
   50,
   52,
   54,
   56,
   58,
   60,
   62,
   64,
   66,
   68,
   70,
   72,
   74,
   76,
   78,
   80,
   82,
   84,
   86,
   88,
   90,
   92,
   94,
   96,
   98,
   100,
   102,
   104,
   106,
   108,
   110,
   112,
   114,
   116,
   118,
   120,
   122,
   124,
   126,
   128,
   130,
   132,
   134,
   136,
   138,
   140,
   142,
   144,
   146,
   148,
   150,
   152,
   154,
   156,
   158,
   160,
   162,
   164,
   166,
   168,
   170,
   172,
   174,
   176,
   178,
   180,
   182,
   184,
   186,
   188,
   190,
   192,
   194,
   196,
   198,
   200,
   202,
   204,
   206,
   208,
   210,
   212,
   214,
   216,
   218,
   220,
   222,
   224,
   226,
   228,
   230,
   232,
   234,
   236,
   238,
   240,
   242,
   244,
   246,
   248,
   250,
   252,